# Querying and Creating Portfolios in VAM Client
This notebook serves as a tutorial for querying, creating, and managing portfolios using the `mainsequence.client` package. It covers:
- Querying existing portfolios using filters
- Creating portfolios from time series
- Creating an index asset portfolio
- Querying the newly created index asset portfolio

## Step 1: Import Required Modules
First, we import the necessary modules and constants.

In [8]:
import os
from pathlib import Path
import dotenv
# Save the original working directory (only once)
try:
    original_wd
except NameError:
    original_wd = os.getcwd()

# Compute the target directory: two levels up from the original working directory
# For example, if original_wd is /Users/username/project/notebooks,
# then target_dir becomes /Users/username
target_dir = Path(original_wd).parents[2]

# Change the working directory to the target directory
os.chdir(target_dir)
print("Working directory set to:", os.getcwd())

import dotenv
dotenv.load_dotenv('.env')
from mainsequence.client import DataUpdates
import datetime
from mainsequence.client import TargetPortfolio, Asset,TargetPortfolioIndexAsset
from mainsequence import MARKETS_CONSTANTS 


Working directory set to: /home/jose/code/MainSequenceClientSide/mainsequence-sdk


## Step 2: Query Portfolios
We query an existing portfolio using the `local_time_serie_hash_id` filter. This returns a single portfolio object.

In [2]:

# Query a single TargetPortfolio with a specific ticker
portfolio_ticker = "TICKER1"
portfolio = TargetPortfolio.filter(portfolio_ticker=portfolio_ticker)
print("Queried Portfolio:", portfolio)


Queried Portfolio: []


## Step 3: Create Portfolios from Time Series
We create two portfolios for different purposes:
- **Execution Portfolio** for real-time execution
- **Backtesting Portfolio** for simulation and historical analysis

### !!! Important !!!

If you want to start building portfolios we recommend you to use our VirtualFundBuilder. This will help you have a more fluid 
and seamless workflows. In this example we will show you how to build a portfolio without using our Wrapper. A few important considerations

1.  Each portfolio requires a `local_signal_time_serie` and a `local_time_serie` that should come from TDAG, this is a unique identifier in the backend and placing a random integer will likely collide therefore we need to build a time serie that has a time_index and at least a column with a column "close" 



In [3]:
from mainsequence.tdag.time_series import TimeSerie
from mainsequence.client import DataUpdates
import numpy as np
import datetime
import pandas as pd

class PortfolioExample(TimeSerie):
    
    @TimeSerie._post_init_routines()
    def __init__(self, portfolio_volatility: float,source_prices:str, *args, **kwargs):
        self.portfolio_volatility = portfolio_volatility
        self.source_prices=source_prices
        super().__init__(*args, **kwargs)
        
    def update(self, update_statistics):
        """
        Update the time series by simulating a new data point based on the last observation,
        or simulate a complete series if no observations exist.

        :param update_statistics: A dict to record update statistics (optional).
        :return: pd.DataFrame with index as a timezone-aware datetime (UTC) and a column "close".
        """
        last_observation = self.get_last_observation()
        dt = 1.0  # one day time increment

        if last_observation is not None:
            return pd.DataFrame() # do not make sequential updats for example
          
        else:
            # If no observation exists, simulate a daily series for the last 30 days.
            end_date = datetime.datetime.now(datetime.timezone.utc)
            start_date = end_date - datetime.timedelta(days=30)
            # Generate a date range with daily frequency (timezone-aware in UTC).
            dates = pd.date_range(start=start_date, end=end_date, freq='D', tz=datetime.timezone.utc)
            
            # Initialize with a default price.
            prices = [100.0]
            for _ in range(1, len(dates)):
                random_return = np.random.normal(loc=0, scale=self.portfolio_volatility * np.sqrt(dt))
                new_price = prices[-1] * np.exp(random_return - 0.5 * self.portfolio_volatility**2 * dt)
                prices.append(new_price)
                
            new_data = pd.DataFrame({"close": prices}, index=dates)
        if last_observation is not None:
            new_data=new_data[new_data.index>update_statistics._max_time_in_update_statistics]
         
        return new_data
    
    
portfolio_ts=PortfolioExample(portfolio_volatility=.1,source_prices="backtest")
portfolio_ts.run(debug_mode=True,force_update=True)


2025-04-10T20:43:11.682553Z [warning  ] Your TimeSeries is not in a python module this will likely bring exceptions when running in a pipeline api_time_series=False application_name=ms-sdk data_source_id=1 job_run_id=None local_hash_id=portfolioexample_f53f240ed0980b8f8bd541172af036ac local_hash_id_data_source=1 project_id=39 (at time_series.py:1770 in get_time_serie_source_code())
2025-04-10T20:43:11.689127Z [warning  ] Your TimeSeries is not in a python module this will likely bring exceptions when running in a pipeline api_time_series=False application_name=ms-sdk data_source_id=1 job_run_id=None local_hash_id=portfolioexample_f53f240ed0980b8f8bd541172af036ac local_hash_id_data_source=1 project_id=39 (at time_series.py:1770 in get_time_serie_source_code())
Overriding of current TracerProvider is not allowed
2025-04-10T20:43:17.500671Z [info     ] Updating Local Time Series for  PortfolioExample http://127.0.0.1:8000/local-time-series/details/?local_time_serie_id=127  for first time 

In [4]:
#we can get our simulated prices from TDAG Backend
print(f"Data in time series",portfolio_ts)
portfolio_ts.get_df_between_dates()

Data in time series PortfolioExample http://127.0.0.1:8000/local-time-series/details/?local_time_serie_id=127


,close
time_index,
2025-03-11 19:21:16.768000+00:00,100.000000
2025-03-11 19:24:06.269000+00:00,100.000000
2025-03-11 19:27:17.342000+00:00,100.000000
2025-03-11 19:28:13.756000+00:00,100.000000
2025-03-11 20:32:54.714000+00:00,100.000000
2025-03-11 20:43:18.084000+00:00,100.000000
2025-03-12 20:43:18.084000+00:00,86.214177
2025-03-13 20:43:18.084000+00:00,100.961729
2025-03-14 20:43:18.084000+00:00,95.929417


In [16]:

def create_portfolio(build_purpose, portfolio_name,portfolio_ts,valuation_asset):
    
    
    existing_portfolio = TargetPortfolio.get_or_none(
       
        local_time_serie__id=portfolio_ts.local_metadata.id
    )
    
    if existing_portfolio:
        print(f"Portfolio '{portfolio_name}' already exists.")
        index_asset=TargetPortfolioIndexAsset.get(reference_portfolio__id=existing_portfolio.id)
        
        
        return existing_portfolio,index_asset
    
    
    return TargetPortfolio.create_from_time_series(
        portfolio_name=portfolio_name,
        build_purpose=build_purpose,
        is_active=True,
        local_time_serie_id=portfolio_ts.local_metadata.id,  # Example ID
        signal_local_time_serie_id=portfolio_ts.local_metadata.id, #we are not using a signal so we are setting the same 
        required_venues__symbols=[MARKETS_CONSTANTS.BINANCE_EV_SYMBOL],
        calendar_name="24/7",
        tracking_funds_expected_exposure_from_latest_holdings=False, 
        is_asset_only=False,
        backtest_table_price_column_name="close",
        valuation_asset_id=valuation_asset.id,
        target_portfolio_about=dict(description= "Test Portfolio For example",
                                    signal_name= "No signal",
                                    signal_description="no descritpiont",
                                    rebalance_strategy_name="no rebalance"), timeout=600000
    )

valuation_asset=Asset.get(symbol="USDT",execution_venue__symbol=MARKETS_CONSTANTS.BINANCE_EV_SYMBOL)
# Create Backtesting Portfolio
backtest_portfolio, portfolio_index_asset = create_portfolio(
    MARKETS_CONSTANTS.PORTFOLIO_BUILD_FOR_BACKTEST, "Example Portfolio",portfolio_ts,valuation_asset,
)
print("Created Backtest Portfolio:", backtest_portfolio.portfolio_name)
print("Related Asset", valuation_asset)


Portfolio 'Example Portfolio' already exists.
Created Backtest Portfolio: Example Portfolio
Related Asset orm_class='Asset' id=192402 symbol='USDT' name='USDT' can_trade=False calendar=Calendar: 1 execution_venue=ExecutionVenue: 9 delisted_datetime=None unique_identifier='USDT_bnce_KKG000000H64' figi_details=FigiInfo(orm_class='FigiInfo', id=111937, real_figi=True, figi='KKG000000H64', composite=None, ticker='USDT', security_type='Crypto', security_type_2='CRYPTO', security_market_sector='Curncy', share_class=None, exchange_code=None, name='tether', main_sequence_share_class='LwZ5U4MvuuIH')
